# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

#Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qa'en,33.7265,59.1844,22.47,30,1,3.74,IR,1691533248
1,1,yellowknife,62.4560,-114.3525,22.29,40,20,7.72,CA,1691533214
2,2,iqaluit,63.7506,-68.5145,5.85,86,40,4.63,CA,1691532923
3,3,waitangi,-43.9535,-176.5597,9.34,92,100,0.89,NZ,1691532903
4,4,constantia,44.1833,28.6500,15.39,66,46,3.79,RO,1691532935


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
# YOUR CODE HERE
map_plot_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_cloudiness = city_data_df.loc[city_data_df["Cloudiness"]==0,:]
ideal_wind = ideal_cloudiness.loc[(ideal_cloudiness["Wind Speed"]<4.5),:]
ideal_temp = ideal_wind.loc[(ideal_wind["Max Temp"]<27),:]
ideal_temp = ideal_temp.loc[(ideal_wind["Max Temp"]>21),:]
# Drop any rows with null values
# YOUR CODE HERE
ideal_temp_df = ideal_temp.dropna()
# Display sample data
# YOUR CODE HERE
ideal_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,vadso,70.0744,29.7487,21.49,94,0,2.57,NO,1691532911
158,158,akcadag,38.3453,37.9672,22.55,38,0,0.51,TR,1691533260
204,204,pervomayskoye,43.9190,46.7102,25.64,58,0,3.47,RU,1691533264
210,210,hammerfest,70.6634,23.6821,23.25,73,0,2.06,NO,1691532934
233,233,rengasdengklok,-6.1592,107.2981,25.07,88,0,1.92,ID,1691533266
247,247,yanykurgan,43.9150,67.2481,26.43,35,0,1.98,KZ,1691533267
365,365,langarud,37.1970,50.1535,26.73,67,0,1.17,IR,1691533281
412,412,tudela,42.0617,-1.6045,24.50,37,0,2.94,ES,1691533286
428,428,anapa,44.8908,37.3239,22.19,83,0,0.00,RU,1691533288
431,431,mahdia,35.5047,11.0622,24.97,60,0,1.28,TN,1691533288


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
ideal_temp_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_temp_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

C:\Users\chang\AppData\Local\Temp\ipykernel_24556\3502102824.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
87,vadso,NO,70.0744,29.7487,94,
158,akcadag,TR,38.3453,37.9672,38,
204,pervomayskoye,RU,43.9190,46.7102,58,
210,hammerfest,NO,70.6634,23.6821,73,
233,rengasdengklok,ID,-6.1592,107.2981,88,
247,yanykurgan,KZ,43.9150,67.2481,35,
365,langarud,IR,37.1970,50.1535,67,
412,tudela,ES,42.0617,-1.6045,37,
428,anapa,RU,44.8908,37.3239,83,
431,mahdia,TN,35.5047,11.0622,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# to make sure API key is working and to check categories
latitude = 40.6000
longitude = -6.5333

# Set the parameters for the type of place
categories = "accommodation.hotel"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"
limit = 1

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
reasponse = requests.get(base_url, params=params).json()
pprint.pprint(reasponse)

{'features': [{'geometry': {'coordinates': [-6.5348708, 40.59846300034493],
                            'type': 'Point'},
               'properties': {'address_line1': 'Hotel Arcos',
                              'address_line2': 'Calle del Cardenal Pacheco, '
                                               '37500 Ciudad Rodrigo, Spain',
                              'categories': ['accommodation',
                                             'accommodation.hotel'],
                              'city': 'Ciudad Rodrigo',
                              'country': 'Spain',
                              'country_code': 'es',
                              'county': 'Salamanca',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database Licence',
                                             'raw': {'name': 'Hotel Arcos',
      

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories": categories,
    "limit":limit,
    "filter": filter,
    "bias": bias,
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    #city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vadso - nearest hotel: Scandic Vadsø
akcadag - nearest hotel: No hotel found
pervomayskoye - nearest hotel: Европа
hammerfest - nearest hotel: Thon Hotel Hammerfest
rengasdengklok - nearest hotel: No hotel found
yanykurgan - nearest hotel: No hotel found
langarud - nearest hotel: هتل رز
tudela - nearest hotel: Hostal Remigio
anapa - nearest hotel: Анапа Лазурная
mahdia - nearest hotel: La Maison Bleue
darwin - nearest hotel: Mantra Pantanas Darwin


,City,Country,Lat,Lng,Humidity,Hotel Name
87,vadso,NO,70.0744,29.7487,94,Scandic Vadsø
158,akcadag,TR,38.3453,37.9672,38,No hotel found
204,pervomayskoye,RU,43.9190,46.7102,58,Европа
210,hammerfest,NO,70.6634,23.6821,73,Thon Hotel Hammerfest
233,rengasdengklok,ID,-6.1592,107.2981,88,No hotel found
247,yanykurgan,KZ,43.9150,67.2481,35,No hotel found
365,langarud,IR,37.1970,50.1535,67,هتل رز
412,tudela,ES,42.0617,-1.6045,37,Hostal Remigio
428,anapa,RU,44.8908,37.3239,83,Анапа Лазурная
431,mahdia,TN,35.5047,11.0622,60,La Maison Bleue


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_Hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
map_plot_Hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)